# TPS 02-2022: Tree Ensembles

In [1]:
# Global variables for testing changes to this notebook quickly
RANDOM_SEED = 0
NUM_FOLDS = 3
NUM_SAMPLES = 50000

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import time
import re
import math
import os
import pyarrow
import gc

# Model evaluation
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_selection import mutual_info_classif


# Plotting
import matplotlib
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import Image

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [3]:
def sample_data(samples):
    
    train = pd.read_feather('../data/train.feather')
    train, _ = train_test_split(train, train_size = samples, stratify = train['target'])
    return train

In [4]:
%%time
from sklearn.preprocessing import LabelEncoder

train = sample_data(NUM_SAMPLES)
features = [x for x in train.columns if x not in ['row_id', 'target']]

# Label encoding
encoder = LabelEncoder()
train['target'] = encoder.fit_transform(train['target'])

print(f'Training Samples: {len(train)}')

Training Samples: 50000
Wall time: 564 ms


# Scoring Function

In [5]:
# Scoring/Training Baseline Function
def score_model(sklearn_model):
    
    # Store the holdout predictions
    oof_preds = np.zeros((train.shape[0],))
    scores = np.zeros(NUM_FOLDS)
    times = np.zeros(NUM_FOLDS)
    print('')
    
    # Stratified k-fold cross-validation
    skf = StratifiedKFold(n_splits = NUM_FOLDS, shuffle = True, random_state = RANDOM_SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['target'])):
        
        # Training and Validation Sets
        X_train, y_train = train[features].iloc[train_idx].to_numpy(), train['target'].iloc[train_idx].to_numpy()
        X_valid, y_valid = train[features].iloc[valid_idx].to_numpy(), train['target'].iloc[valid_idx].to_numpy()
        
        # Create model
        model = clone(sklearn_model)
            
        start = time.time()

        model.fit(X_train, y_train)
        
        end = time.time()
        
        # validation
        try:
            valid_preds = model.predict_proba(X_valid)
            scores[fold] = accuracy_score(y_valid, np.argmax(valid_preds, axis = 1))
        except:
            scores[fold] = accuracy_score(y_valid, model.predict(X_valid))
        end = time.time()
        print(f'Fold {fold}: {round(scores[fold], 5)} accuracy in {round(end-start,2)}s.')
        times[fold] = end-start
        
        
    print("\nAverage Accuracy:", round(scores.mean(), 5))
    print(f'Training Time: {round(times.sum(), 2)}s')

# Sklearn Tree Ensembles

In [6]:
# Base Estimators
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

## 1. AdaBoost

In [7]:
from sklearn.ensemble import AdaBoostClassifier

In [8]:
# Defaults
score_model(AdaBoostClassifier())


Fold 0: 0.32669 accuracy in 15.8s.
Fold 1: 0.28415 accuracy in 15.81s.
Fold 2: 0.32353 accuracy in 15.79s.

Average Accuracy: 0.31146
Training Time: 47.4s


In [9]:
# Base Estimator - Default DecisionTree
score_model(
    AdaBoostClassifier(
        DecisionTreeClassifier()
    )
)


Fold 0: 0.85846 accuracy in 3.97s.
Fold 1: 0.85318 accuracy in 3.86s.
Fold 2: 0.85089 accuracy in 3.95s.

Average Accuracy: 0.85418
Training Time: 11.78s


In [10]:
# Base Estimator - ExtraTrees
score_model(
    AdaBoostClassifier(
        ExtraTreeClassifier()
    )
)


Fold 0: 0.82972 accuracy in 0.16s.
Fold 1: 0.82618 accuracy in 0.17s.
Fold 2: 0.83157 accuracy in 0.16s.

Average Accuracy: 0.82916
Training Time: 0.49s


## 2. Bagging Classifier

In [11]:
from sklearn.ensemble import BaggingClassifier

In [12]:
score_model(BaggingClassifier())


Fold 0: 0.90172 accuracy in 24.82s.
Fold 1: 0.90022 accuracy in 24.43s.
Fold 2: 0.9007 accuracy in 26.26s.

Average Accuracy: 0.90088
Training Time: 75.51s


In [13]:
score_model(
    BaggingClassifier(
        DecisionTreeClassifier()
    )
)


Fold 0: 0.90646 accuracy in 26.03s.
Fold 1: 0.90544 accuracy in 24.68s.
Fold 2: 0.90082 accuracy in 25.7s.

Average Accuracy: 0.90424
Training Time: 76.4s


In [14]:
score_model(
    BaggingClassifier(
        ExtraTreeClassifier()
    )
)


Fold 0: 0.89656 accuracy in 1.01s.
Fold 1: 0.8953 accuracy in 1.01s.
Fold 2: 0.89434 accuracy in 1.01s.

Average Accuracy: 0.8954
Training Time: 3.03s


## 3. Extra Trees

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

In [16]:
score_model(ExtraTreesClassifier(n_jobs = -1))


Fold 0: 0.95308 accuracy in 2.82s.
Fold 1: 0.94768 accuracy in 2.8s.
Fold 2: 0.94876 accuracy in 2.82s.

Average Accuracy: 0.94984
Training Time: 8.45s


## 4. Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
score_model(RandomForestClassifier())


Fold 0: 0.95152 accuracy in 15.81s.
Fold 1: 0.94516 accuracy in 15.89s.
Fold 2: 0.94492 accuracy in 16.11s.

Average Accuracy: 0.9472
Training Time: 47.82s


## 5. Gradient Boosting

In [19]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [20]:
score_model(HistGradientBoostingClassifier())


Fold 0: 0.94834 accuracy in 68.3s.
Fold 1: 0.94414 accuracy in 67.15s.
Fold 2: 0.9439 accuracy in 66.9s.

Average Accuracy: 0.94546
Training Time: 202.35s


# XGBoost

In [21]:
from xgboost import XGBClassifier

In [22]:
score_model(XGBClassifier(eval_metric = 'mlogloss'))


Fold 0: 0.95176 accuracy in 51.12s.
Fold 1: 0.946 accuracy in 50.9s.
Fold 2: 0.94702 accuracy in 51.14s.

Average Accuracy: 0.94826
Training Time: 153.16s


# LightGBM

In [23]:
from lightgbm import LGBMClassifier

In [24]:
score_model(LGBMClassifier())


Fold 0: 0.95068 accuracy in 11.71s.
Fold 1: 0.94798 accuracy in 11.66s.
Fold 2: 0.94678 accuracy in 11.66s.

Average Accuracy: 0.94848
Training Time: 35.03s


# CatBoost

In [25]:
from catboost import CatBoostClassifier

In [26]:
score_model(CatBoostClassifier())


Learning rate set to 0.094622
0:	learn: 2.0658875	total: 567ms	remaining: 9m 26s
1:	learn: 1.8981102	total: 1.01s	remaining: 8m 23s
2:	learn: 1.7820241	total: 1.44s	remaining: 7m 57s
3:	learn: 1.6862169	total: 1.86s	remaining: 7m 43s
4:	learn: 1.6023411	total: 2.29s	remaining: 7m 36s
5:	learn: 1.5384433	total: 2.72s	remaining: 7m 30s
6:	learn: 1.4760527	total: 3.15s	remaining: 7m 27s
7:	learn: 1.4229460	total: 3.58s	remaining: 7m 23s
8:	learn: 1.3699548	total: 4s	remaining: 7m 20s
9:	learn: 1.3241401	total: 4.43s	remaining: 7m 18s
10:	learn: 1.2859039	total: 4.87s	remaining: 7m 17s
11:	learn: 1.2479655	total: 5.29s	remaining: 7m 15s
12:	learn: 1.2115916	total: 5.72s	remaining: 7m 14s
13:	learn: 1.1788221	total: 6.16s	remaining: 7m 13s
14:	learn: 1.1511122	total: 6.58s	remaining: 7m 12s
15:	learn: 1.1277452	total: 7.04s	remaining: 7m 12s
16:	learn: 1.1026692	total: 7.46s	remaining: 7m 11s
17:	learn: 1.0794826	total: 7.89s	remaining: 7m 10s
18:	learn: 1.0570552	total: 8.32s	remaining: 7

158:	learn: 0.4443207	total: 1m 7s	remaining: 5m 56s
159:	learn: 0.4429186	total: 1m 7s	remaining: 5m 55s
160:	learn: 0.4414121	total: 1m 8s	remaining: 5m 55s
161:	learn: 0.4398410	total: 1m 8s	remaining: 5m 54s
162:	learn: 0.4386635	total: 1m 8s	remaining: 5m 54s
163:	learn: 0.4378162	total: 1m 9s	remaining: 5m 53s
164:	learn: 0.4368837	total: 1m 9s	remaining: 5m 53s
165:	learn: 0.4356245	total: 1m 10s	remaining: 5m 52s
166:	learn: 0.4340650	total: 1m 10s	remaining: 5m 52s
167:	learn: 0.4325246	total: 1m 11s	remaining: 5m 51s
168:	learn: 0.4306313	total: 1m 11s	remaining: 5m 51s
169:	learn: 0.4299675	total: 1m 11s	remaining: 5m 50s
170:	learn: 0.4287936	total: 1m 12s	remaining: 5m 50s
171:	learn: 0.4269118	total: 1m 12s	remaining: 5m 49s
172:	learn: 0.4255230	total: 1m 13s	remaining: 5m 49s
173:	learn: 0.4239920	total: 1m 13s	remaining: 5m 49s
174:	learn: 0.4228063	total: 1m 13s	remaining: 5m 48s
175:	learn: 0.4211081	total: 1m 14s	remaining: 5m 48s
176:	learn: 0.4197358	total: 1m 14s

311:	learn: 0.3088491	total: 2m 10s	remaining: 4m 48s
312:	learn: 0.3084909	total: 2m 11s	remaining: 4m 47s
313:	learn: 0.3079710	total: 2m 11s	remaining: 4m 47s
314:	learn: 0.3076062	total: 2m 11s	remaining: 4m 46s
315:	learn: 0.3072259	total: 2m 12s	remaining: 4m 46s
316:	learn: 0.3062854	total: 2m 12s	remaining: 4m 46s
317:	learn: 0.3058200	total: 2m 13s	remaining: 4m 45s
318:	learn: 0.3054698	total: 2m 13s	remaining: 4m 45s
319:	learn: 0.3049700	total: 2m 14s	remaining: 4m 44s
320:	learn: 0.3045192	total: 2m 14s	remaining: 4m 44s
321:	learn: 0.3034977	total: 2m 14s	remaining: 4m 43s
322:	learn: 0.3028133	total: 2m 15s	remaining: 4m 43s
323:	learn: 0.3022605	total: 2m 15s	remaining: 4m 43s
324:	learn: 0.3018018	total: 2m 16s	remaining: 4m 42s
325:	learn: 0.3013960	total: 2m 16s	remaining: 4m 42s
326:	learn: 0.3007799	total: 2m 16s	remaining: 4m 41s
327:	learn: 0.2999249	total: 2m 17s	remaining: 4m 41s
328:	learn: 0.2996198	total: 2m 17s	remaining: 4m 40s
329:	learn: 0.2989734	total:

464:	learn: 0.2411055	total: 3m 14s	remaining: 3m 43s
465:	learn: 0.2407754	total: 3m 14s	remaining: 3m 42s
466:	learn: 0.2403528	total: 3m 14s	remaining: 3m 42s
467:	learn: 0.2399842	total: 3m 15s	remaining: 3m 41s
468:	learn: 0.2396993	total: 3m 15s	remaining: 3m 41s
469:	learn: 0.2393587	total: 3m 16s	remaining: 3m 41s
470:	learn: 0.2391456	total: 3m 16s	remaining: 3m 40s
471:	learn: 0.2387198	total: 3m 16s	remaining: 3m 40s
472:	learn: 0.2384590	total: 3m 17s	remaining: 3m 39s
473:	learn: 0.2383105	total: 3m 17s	remaining: 3m 39s
474:	learn: 0.2381093	total: 3m 18s	remaining: 3m 38s
475:	learn: 0.2378175	total: 3m 18s	remaining: 3m 38s
476:	learn: 0.2375557	total: 3m 18s	remaining: 3m 38s
477:	learn: 0.2373186	total: 3m 19s	remaining: 3m 37s
478:	learn: 0.2371893	total: 3m 19s	remaining: 3m 37s
479:	learn: 0.2367383	total: 3m 20s	remaining: 3m 36s
480:	learn: 0.2364913	total: 3m 20s	remaining: 3m 36s
481:	learn: 0.2361488	total: 3m 21s	remaining: 3m 36s
482:	learn: 0.2359640	total:

617:	learn: 0.1984519	total: 4m 17s	remaining: 2m 39s
618:	learn: 0.1982241	total: 4m 17s	remaining: 2m 38s
619:	learn: 0.1978915	total: 4m 18s	remaining: 2m 38s
620:	learn: 0.1976907	total: 4m 18s	remaining: 2m 37s
621:	learn: 0.1974019	total: 4m 18s	remaining: 2m 37s
622:	learn: 0.1969796	total: 4m 19s	remaining: 2m 36s
623:	learn: 0.1968931	total: 4m 19s	remaining: 2m 36s
624:	learn: 0.1966765	total: 4m 20s	remaining: 2m 36s
625:	learn: 0.1965071	total: 4m 20s	remaining: 2m 35s
626:	learn: 0.1962828	total: 4m 20s	remaining: 2m 35s
627:	learn: 0.1959057	total: 4m 21s	remaining: 2m 34s
628:	learn: 0.1956444	total: 4m 21s	remaining: 2m 34s
629:	learn: 0.1952953	total: 4m 22s	remaining: 2m 33s
630:	learn: 0.1949043	total: 4m 22s	remaining: 2m 33s
631:	learn: 0.1946552	total: 4m 23s	remaining: 2m 33s
632:	learn: 0.1943026	total: 4m 23s	remaining: 2m 32s
633:	learn: 0.1941289	total: 4m 23s	remaining: 2m 32s
634:	learn: 0.1936892	total: 4m 24s	remaining: 2m 31s
635:	learn: 0.1934784	total:

...

317:	learn: 0.3014629	total: 2m 13s	remaining: 4m 46s
318:	learn: 0.3008969	total: 2m 13s	remaining: 4m 45s
319:	learn: 0.3002693	total: 2m 14s	remaining: 4m 45s
320:	learn: 0.2997210	total: 2m 14s	remaining: 4m 45s
321:	learn: 0.2993430	total: 2m 15s	remaining: 4m 44s
322:	learn: 0.2989420	total: 2m 15s	remaining: 4m 44s
323:	learn: 0.2984938	total: 2m 16s	remaining: 4m 43s
324:	learn: 0.2978236	total: 2m 16s	remaining: 4m 43s
325:	learn: 0.2970832	total: 2m 16s	remaining: 4m 42s
326:	learn: 0.2964826	total: 2m 17s	remaining: 4m 42s
327:	learn: 0.2961692	total: 2m 17s	remaining: 4m 42s
328:	learn: 0.2955166	total: 2m 18s	remaining: 4m 41s
329:	learn: 0.2947936	total: 2m 18s	remaining: 4m 41s
330:	learn: 0.2943268	total: 2m 18s	remaining: 4m 40s
331:	learn: 0.2934074	total: 2m 19s	remaining: 4m 40s
332:	learn: 0.2929671	total: 2m 19s	remaining: 4m 40s
333:	learn: 0.2923200	total: 2m 20s	remaining: 4m 39s
334:	learn: 0.2919010	total: 2m 20s	remaining: 4m 39s
335:	learn: 0.2912592	total:

470:	learn: 0.2343285	total: 3m 17s	remaining: 3m 41s
471:	learn: 0.2339178	total: 3m 17s	remaining: 3m 40s
472:	learn: 0.2336771	total: 3m 17s	remaining: 3m 40s
473:	learn: 0.2333899	total: 3m 18s	remaining: 3m 40s
474:	learn: 0.2329990	total: 3m 18s	remaining: 3m 39s
475:	learn: 0.2327553	total: 3m 19s	remaining: 3m 39s
476:	learn: 0.2325482	total: 3m 19s	remaining: 3m 38s
477:	learn: 0.2322889	total: 3m 19s	remaining: 3m 38s
478:	learn: 0.2318746	total: 3m 20s	remaining: 3m 37s
479:	learn: 0.2316505	total: 3m 20s	remaining: 3m 37s
480:	learn: 0.2313753	total: 3m 21s	remaining: 3m 37s
481:	learn: 0.2309559	total: 3m 21s	remaining: 3m 36s
482:	learn: 0.2305872	total: 3m 22s	remaining: 3m 36s
483:	learn: 0.2302921	total: 3m 22s	remaining: 3m 35s
484:	learn: 0.2299828	total: 3m 22s	remaining: 3m 35s
485:	learn: 0.2294777	total: 3m 23s	remaining: 3m 34s
486:	learn: 0.2291962	total: 3m 23s	remaining: 3m 34s
487:	learn: 0.2289032	total: 3m 24s	remaining: 3m 34s
488:	learn: 0.2286073	total:

623:	learn: 0.1918084	total: 4m 20s	remaining: 2m 37s
624:	learn: 0.1916636	total: 4m 21s	remaining: 2m 36s
625:	learn: 0.1915288	total: 4m 21s	remaining: 2m 36s
626:	learn: 0.1913947	total: 4m 22s	remaining: 2m 35s
627:	learn: 0.1910765	total: 4m 22s	remaining: 2m 35s
628:	learn: 0.1908980	total: 4m 22s	remaining: 2m 35s
629:	learn: 0.1907332	total: 4m 23s	remaining: 2m 34s
630:	learn: 0.1904475	total: 4m 23s	remaining: 2m 34s
631:	learn: 0.1903011	total: 4m 24s	remaining: 2m 33s
632:	learn: 0.1901487	total: 4m 24s	remaining: 2m 33s
633:	learn: 0.1899398	total: 4m 25s	remaining: 2m 33s
634:	learn: 0.1895360	total: 4m 25s	remaining: 2m 32s
635:	learn: 0.1893992	total: 4m 25s	remaining: 2m 32s
636:	learn: 0.1892924	total: 4m 26s	remaining: 2m 31s
637:	learn: 0.1891628	total: 4m 26s	remaining: 2m 31s
638:	learn: 0.1889681	total: 4m 27s	remaining: 2m 30s
639:	learn: 0.1887518	total: 4m 27s	remaining: 2m 30s
640:	learn: 0.1886285	total: 4m 28s	remaining: 2m 30s
641:	learn: 0.1884691	total:

776:	learn: 0.1626587	total: 5m 24s	remaining: 1m 33s
777:	learn: 0.1624084	total: 5m 24s	remaining: 1m 32s
778:	learn: 0.1623015	total: 5m 25s	remaining: 1m 32s
779:	learn: 0.1621973	total: 5m 25s	remaining: 1m 31s
780:	learn: 0.1619924	total: 5m 26s	remaining: 1m 31s
781:	learn: 0.1618882	total: 5m 26s	remaining: 1m 31s
782:	learn: 0.1617058	total: 5m 26s	remaining: 1m 30s
783:	learn: 0.1615149	total: 5m 27s	remaining: 1m 30s
784:	learn: 0.1613457	total: 5m 27s	remaining: 1m 29s
785:	learn: 0.1612629	total: 5m 28s	remaining: 1m 29s
786:	learn: 0.1611343	total: 5m 28s	remaining: 1m 28s
787:	learn: 0.1609614	total: 5m 29s	remaining: 1m 28s
788:	learn: 0.1608357	total: 5m 29s	remaining: 1m 28s
789:	learn: 0.1605864	total: 5m 29s	remaining: 1m 27s
790:	learn: 0.1604229	total: 5m 30s	remaining: 1m 27s
791:	learn: 0.1603008	total: 5m 30s	remaining: 1m 26s
792:	learn: 0.1600124	total: 5m 31s	remaining: 1m 26s
793:	learn: 0.1598683	total: 5m 31s	remaining: 1m 26s
794:	learn: 0.1597809	total:

931:	learn: 0.1403323	total: 6m 28s	remaining: 28.4s
932:	learn: 0.1401431	total: 6m 29s	remaining: 28s
933:	learn: 0.1400393	total: 6m 29s	remaining: 27.5s
934:	learn: 0.1399626	total: 6m 30s	remaining: 27.1s
935:	learn: 0.1398232	total: 6m 30s	remaining: 26.7s
936:	learn: 0.1397035	total: 6m 30s	remaining: 26.3s
937:	learn: 0.1396359	total: 6m 31s	remaining: 25.9s
938:	learn: 0.1395161	total: 6m 31s	remaining: 25.5s
939:	learn: 0.1394390	total: 6m 32s	remaining: 25s
940:	learn: 0.1392165	total: 6m 32s	remaining: 24.6s
941:	learn: 0.1391442	total: 6m 33s	remaining: 24.2s
942:	learn: 0.1389818	total: 6m 33s	remaining: 23.8s
943:	learn: 0.1388357	total: 6m 33s	remaining: 23.4s
944:	learn: 0.1387596	total: 6m 34s	remaining: 22.9s
945:	learn: 0.1387005	total: 6m 34s	remaining: 22.5s
946:	learn: 0.1385075	total: 6m 35s	remaining: 22.1s
947:	learn: 0.1384529	total: 6m 35s	remaining: 21.7s
948:	learn: 0.1383396	total: 6m 35s	remaining: 21.3s
949:	learn: 0.1382528	total: 6m 36s	remaining: 20.